In [44]:
import datetime
import numpy as np
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model
from sklearn import preprocessing
from sklearn import neighbors as knn

In [2]:
data = pd.read_csv("extendedData.csv").dropna()
data.reset_index(drop=True, inplace=True)
data = data.drop('Unnamed: 0', 1)
data = data.drop('address', 1)
data = data.drop('id', 1)
data = data.drop('name', 1)
data = data.drop('collection_id', 1)

In [3]:
for j, k in enumerate(data["DOB"]):
    data.at[j, "DOB"] = datetime.datetime.now().year - int(data.at[j, "DOB"][:4])
data.rename(columns={'DOB' : 'age'}, inplace=True)

In [4]:
le = preprocessing.LabelEncoder()

In [5]:
le.fit(data['city'])
data['city'] = le.transform(data['city'])
le.fit(data['sex'])
data['sex'] = le.transform(data['sex'])
le.fit(data['state'])
data['state'] = le.transform(data['state'])
le.fit(data['MARITAL_STATUS'])
data['MARITAL_STATUS'] = le.transform(data['MARITAL_STATUS'])
le.fit(data['TOBACCO'])
data['TOBACCO'] = le.transform(data['TOBACCO'])
data['PURCHASED'] = ["None" if type(i)==float else i for i in list(data["PURCHASED"])]
le.fit(data['PURCHASED'])
data['PURCHASED'] = le.transform(data['PURCHASED'])

In [29]:
data = data.apply(pd.to_numeric, errors='ignore')
data.columns

Index(['city', 'age', 'longitude', 'sex', 'state', 'latitude',
       'PEOPLE_COVERED', 'OPTIONAL_INSURED', 'ANNUAL_INCOME', 'MARITAL_STATUS',
       'HEIGHT', 'WEIGHT', 'TOBACCO', 'GOLD', 'BRONZE', 'SILVER', 'PLATINUM',
       'PURCHASED', 'S62.308', 'G80.4', 'R00.8', 'T85.622', 'Z91.010',
       'T84.011', 'R04.2', 'G47.33', 'R00.0', 'R19.7', 'E11.65', 'B20.1',
       'M05.10', 'F14.121', 'F10.121', 'N18.9'],
      dtype='object')

In [65]:
model = knn.NearestNeighbors(n_neighbors=5)#, metric="mahalanobis", n_jobs=2)
model.fit(data[:40000])

ValueError: Must provide either V or VI for Mahalanobis distance

In [47]:
remaining_data = data[40000:] #random split, normalize data
remaining_data.reset_index(drop=True, inplace=True)

In [63]:
#closest = model.kneighbors(data.head(), 5)
model.kneighbors()

(array([[  33.48133809,   38.44476557,   43.72642222,   58.67708241,
           61.34329629],
        [  68.98550572,  110.28599186,  119.94165248,  143.7810836 ,
          148.56648343],
        [  66.60330322,   70.81666471,   97.17509969,  133.77219442,
          142.46052085],
        ..., 
        [  61.20457499,   67.5203673 ,   75.98026059,   77.06490771,
           78.49203781],
        [  45.88027899,   75.69015788,  108.67842472,  109.73149047,
          112.31651704],
        [  13.45362405,   17.17556404,   19.62141687,   21.23676058,
           22.75961335]]), array([[20746, 22309, 20836, 25407, 35688],
        [19879, 23120, 39421, 28977,  1539],
        [ 7189, 10154, 19150, 31476,  5789],
        ..., 
        [16087, 30236, 19634, 15700, 18616],
        [20437, 38438, 20737, 20541, 36252],
        [ 1804,  7315, 23662, 21273, 10029]], dtype=int64))

In [36]:
Input(shape=(10,))

x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)

NameError: name 'labels' is not defined